# **TRADUCCIÓN DE TEXTO**

> Elena Gómez\
Ana Muñoz





# **DATOS**

Para este proyecto nuestro objetivo es poder crear un modelo generativo de texto que sea capaz de **traducir** texto de entrada en inglés, al español

Encontramos un conjunto de datos en Kaggle que recoge biografías traducidas de Wikipedia del inglés al español.\
Link al dataset: 
https://www.kaggle.com/datasets/paultimothymooney/translated-wikipedia-biographies?select=Translated+Wikipedia+Biographies+-+EN_ES.csv

In [ ]:
import keras
#keras.version_
import pandas as pd
df = pd.read_csv('datos.csv')

In [ ]:
df.head()

,sourceLanguage,targetLanguage,documentID,stringID,sourceText,translatedText,perceivedGender,entityName,sourceURL
0,en,es,1,1-1,Kaisa-Leena Mäkäräinen (born 11 January 1983) ...,Kaisa-Leena Mäkäräinen (nacida el 11 de enero ...,Female,Kaisa Mäkäräinen,https://en.wikipedia.org/wiki/Kaisa_M%C3%A4k%C...
1,en,es,1,1-2,"Outside sports, Mäkäräinen is currently studyi...","Además de los deportes, estudia actualmente en...",Female,Kaisa Mäkäräinen,https://en.wikipedia.org/wiki/Kaisa_M%C3%A4k%C...
2,en,es,1,1-3,"Her team coach is Jonne Kähkönen, while Jarmo ...","El entrenador de su equipo es Jonne Kähkönen, ...",Female,Kaisa Mäkäräinen,https://en.wikipedia.org/wiki/Kaisa_M%C3%A4k%C...
3,en,es,1,1-4,Mäkäräinen was originally a cross-country skie...,Mäkäräinen era originalmente esquiadora de cam...,Female,Kaisa Mäkäräinen,https://en.wikipedia.org/wiki/Kaisa_M%C3%A4k%C...
4,en,es,1,1-5,She started training for the biathlon in 2003.,Comenzó a entrenar para el biatlón en 2003.,Female,Kaisa Mäkäräinen,https://en.wikipedia.org/wiki/Kaisa_M%C3%A4k%C...


De todo el dataset seleccionamos únicamente las columnas que son significativas para nuestro proyecto, estas son **sourceText** que contiene la biografía en inglés y **translatedText** donde se recoge su traducción

In [ ]:
df1 = df[['sourceText','translatedText']]

In [ ]:
df1

,sourceText,translatedText
0,Kaisa-Leena Mäkäräinen (born 11 January 1983) ...,Kaisa-Leena Mäkäräinen (nacida el 11 de enero ...
1,"Outside sports, Mäkäräinen is currently studyi...","Además de los deportes, estudia actualmente en..."
2,"Her team coach is Jonne Kähkönen, while Jarmo ...","El entrenador de su equipo es Jonne Kähkönen, ..."
3,Mäkäräinen was originally a cross-country skie...,Mäkäräinen era originalmente esquiadora de cam...
4,She started training for the biathlon in 2003.,Comenzó a entrenar para el biatlón en 2003.
...,...,...
1466,Speaking to Madrid-based Diario AS in 2013 abo...,"En 2013, habló de sus primeras frustraciones c..."
1467,Rossell proceeded to try again first under San...,Rossell volvió a intentarlo con el sucesor de ...
1468,"In the documentary ""Un Sueño Real"", she reveal...","En el documental «Un sueño real», contó que se..."
1469,Her struggle proved unsuccessful.,Su intento fue en vano.


Procedemos ahora a añadirle los tokens **[start]** y **[end]** a los textos en español, así indicamos donde se inicia la secuencia y donde debe acabar para después traducir.

In [ ]:
df1['translatedText'] = df1['translatedText'].apply(lambda x: "[start] " + x + " [end]")


<ipython-input-5-83d8202e2090>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['translatedText'] = df1['translatedText'].apply(lambda x: "[start] " + x + " [end]")


In [ ]:
df1

,sourceText,translatedText
0,Kaisa-Leena Mäkäräinen (born 11 January 1983) ...,[start] Kaisa-Leena Mäkäräinen (nacida el 11 d...
1,"Outside sports, Mäkäräinen is currently studyi...","[start] Además de los deportes, estudia actual..."
2,"Her team coach is Jonne Kähkönen, while Jarmo ...",[start] El entrenador de su equipo es Jonne Kä...
3,Mäkäräinen was originally a cross-country skie...,[start] Mäkäräinen era originalmente esquiador...
4,She started training for the biathlon in 2003.,[start] Comenzó a entrenar para el biatlón en ...
...,...,...
1466,Speaking to Madrid-based Diario AS in 2013 abo...,"[start] En 2013, habló de sus primeras frustra..."
1467,Rossell proceeded to try again first under San...,[start] Rossell volvió a intentarlo con el suc...
1468,"In the documentary ""Un Sueño Real"", she reveal...","[start] En el documental «Un sueño real», cont..."
1469,Her struggle proved unsuccessful.,[start] Su intento fue en vano. [end]


Antes de hacer las divisiones para los subconjuntos de entrenamiento, validación y prueba, mezclamos los registros del dataset de forma aleatoria

In [ ]:
#Mezclamos de forma aleatoria el dataset
import random
filas = list(df1.index)
random.shuffle(filas)
df1 = df1.loc[filas]

Una vez mezclado, dividimos el conjunto de forma que el conjunto de entrenamiento será el 70% del total, el de validación un 15% y el de test el 15% restante.

In [ ]:
# 15% validación
num_val_samples = int(0.15 * len(df1))
# 70% entrenamiento
num_train_samples = len(df1) - 2 * num_val_samples

train_pairs = df1[:num_train_samples]
val_pairs = df1[num_train_samples:num_train_samples + num_val_samples]

# 15% test o pruebas
test_pairs = df1[num_train_samples + num_val_samples:]

In [ ]:
train_pairs

,sourceText,translatedText
802,She has been a banking chief executive in her ...,[start] Se ha desempeñado como directora ejecu...
594,Her Habilitation in philosophy was completed a...,[start] Obtuvo su habilitación en filosofía en...
151,"As a professional dancer and choreographer, sh...",[start] Como bailarina y coreógrafa profesiona...
614,She has won multiple awards for her work on br...,[start] Recibió varios reconocimientos por su ...
226,He created over 40 works for his new company a...,[start] Ha creado más de 40 obras para su nuev...
...,...,...
806,"Following education both in Zambia and abroad,...",[start] Después de formarse en Zambia y en el ...
1236,She was the first woman with this profession i...,[start] Fue la primera mujer que ejerció esta ...
1382,The band has been voted by fans across the wor...,[start] Sus seguidores de todo el mundo votaro...
1383,"During their ""Best Xmas 2018 Concert"" at Akasa...",[start] Durante el «Best Xmas 2018 Concert» en...


# TextVectorization

Ahora añadimos la capa de TextVectorization.\
Como la forma del texto en español es diferente a la del inglés, debemos preparar dos capas separadas, personalizadas para cada idioma.

En cuanto al texto en español, debemos conservar los tokens añadidos anteriormente ([start] y [end]). Por lo tanto no podemos eliminar los caracteres "[", "]".Si no se especifica se eliminarían por defecto.\
Si se eliminasen, no podríamos distinguir la palabra "start" en inglés del token "[start]".\
En nuestro caso, dado que tenemos un conjunto de datos pequeño, facilitaremos el proceso eliminando los signos de puntuación, sería más eficaz y obtendríamos mejores resultados, por lo menos más realistas y coherentes, si entrenasemos un modelo con signos de puntuación incluidos.


En general, a modo de resumen, lo que hacemos al aplicar la capa TextVectorization es:


1.   Estandarizar, por ejemplo convertir todo el texto a minusculas y eliminar los signos de puntuación
2.   Tokenización: dividir el texto en tokens o unidades
3. Indexación: convertir cada token en un vector numérico



In [ ]:
import tensorflow as tf
import string
import re
from tensorflow.keras import layers

# Para la capa TextVectorization en 
# español: conserva [ y ] pero elimina ¿ (así como 
# todos los demás caracteres de cadenas.puntuación)
strip_chars = string.punctuation + "¿"
print(strip_chars)
strip_chars = strip_chars.replace("[", "")
print(strip_chars)
strip_chars = strip_chars.replace("]", "")
print(strip_chars)

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

# Nos quedamos con las 15.000 palabras principales en cada idioma y 
# restringiremos las oraciones a 30 palabras.
vocab_size = 15000
sequence_length = 30

# La capa en Inglés
source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
# La capa en Español
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    # Generamos oraciones en español que tengan un token 
    # adicional, ya que necesitaremos compensar la oración 
    # en un paso durante el entrenamiento
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = train_pairs['sourceText']
train_spanish_texts = train_pairs['translatedText']
# Aprende el vocabulario de cada idioma
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~¿
!"#$%&'()*+,-./:;<=>?@\]^_`{|}~¿
!"#$%&'()*+,-./:;<=>?@\^_`{|}~¿


In [ ]:
train_spanish_texts

802     [start] Se ha desempeñado como directora ejecu...
594     [start] Obtuvo su habilitación en filosofía en...
151     [start] Como bailarina y coreógrafa profesiona...
614     [start] Recibió varios reconocimientos por su ...
226     [start] Ha creado más de 40 obras para su nuev...
                              ...                        
806     [start] Después de formarse en Zambia y en el ...
1236    [start] Fue la primera mujer que ejerció esta ...
1382    [start] Sus seguidores de todo el mundo votaro...
1383    [start] Durante el «Best Xmas 2018 Concert» en...
811     [start] Fue vicepresidente del Banco de Desarr...
Name: translatedText, Length: 1031, dtype: object

In [ ]:
print(random.choice(train_spanish_texts))

[start] A partir de 1990, fue director del Centro Panruso de Cirugía Plástica y Oftálmica (Ufa). [end]


In [ ]:
batch_size = 32

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        # La oración de entrada en español 
        # no incluye el último token para 
        # mantener las entradas y los 
        # objetivos en la misma longitud.
        "spanish": spa[:, :-1],
    # La frase objetivo en español está un 
    # paso por delante. Ambos siguen siendo 
    # de la misma longitud (20 palabras)
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts = pairs.iloc[:, 0]     #1ª columna (sourceText)
    spa_texts = pairs.iloc[:, 1]     #2ª columna (translatedText)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    # Utilizamos el almacenamiento en caché en memoria 
    # para acelerar el preprocesamiento
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

Así es como se ven los resultados de nuestro conjunto de datos

In [ ]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (32, 30)
inputs['spanish'].shape: (32, 30)
targets.shape: (32, 30)


## **FUNCIONES**

Una vez tenemos los datos listos, vamos a definir e implementar varias funciones útiles y necesarias para nuestro modelo Transformer

## Positional Embedding

El Transformer es un enfoque agnóstico de orden, pero inyecta cierta información del orden manualmente en las representaciones que procesa, a esto lo conocemos como **Positional Encoding**

Para dotar al modelo de la información del orden de las palabras, agregamos la posición de la palabra en la oración a cada word-embedding.\
Ahora, las word-embeddings tendrán dos componentes: el vector de palabra, que representa la palabra independiente del contexto, y un vector de posición, que indica la posición de la palabra en la oración actual.

Aplicando **Positional Embeddings** (incrustación posicional) conseguimos aprender vectores de embedding de posición de la misma manera que aprendimos a incrustar los índices de palabras.\
Se agregan los embeddings de posición a las word-embeddings correspondientes, para así obtener una word-embedding consciente de la posición



In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

## Codificador y Decodificador


**CODIFICADOR TRANSFORMER**\
El codificador del Transformer lee la secuencia original y produce una representación de la misma codificada.\
Este codificador mantiene la representación en un formato de secuencia, es decir, una secuencia de vectores de embeddings conscientes del contexto.

**DECODIFICADOR TRANSFORMER**\
Las partes internas del decodificador se parece mucho al codificador de Transformer, excepto que se inserta un bloque de attention adicional entre el bloque de self-attention aplicado a la secuencia de destino y las capas densas del bloque de salida.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        # Tamaño de los vectores de los tokens de entrada
        self.embed_dim = embed_dim
        # Tamaño de la capa densa interna
        self.dense_dim = dense_dim
        # Número de attention heads
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    # El cálculo va en call()
    def call(self, inputs, mask=None):
        # La máscara que generará la capa Embedding 
        # será 2D, pero la capa de atención espera 
        # ser 3D o 4D, por lo que ampliamos su rango
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    # Implementamos la serialización para 
    # que podamos guardar el modelo
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

La capa **``LayerNormalization``** normaliza cada secuencia independientemente de otras secuencias en el lote, es decir, agrupa los datos dentro de cada secuencia por separadoa.

In [ ]:
# Forma de la entrada: (batch_size, sequence_length, embedding_dim)
def layer_normalization(batch_of_sequences):
    # Para calcular la media y la varianza, solo 
    # agrupamos datos sobre el último eje (eje -1)
    mean = np.mean(batch_of_sequences, keepdims=True, axis=-1)
    variance = np.var(batch_of_sequences, keepdims=True, axis=-1)
    return (batch_of_sequences - mean) / variance

In [ ]:
from tensorflow.keras import layers

In [ ]:
# Forma de la entrada: (batch_size, height, width, channels)
def batch_normalization(batch_of_images):
    # Agrupa los datos sobre el eje del lote (eje 0), 
    # lo que crea interacciones entre las muestras en un lote.
    mean = np.mean(batch_of_images, keepdims=True, axis=(0, 1, 2))
    variance = np.var(batch_of_images, keepdims=True, axis=(0, 1, 2))
    return (batch_of_images - mean) / variance

In [ ]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        # Este atributo asegura que la capa propagará 
        # su máscara de entrada a sus salidas; el 
        # enmascaramiento en Keras es explícitamente 
        # opt-in. Si pasa una máscara a una capa que 
        # no implementa compute_mask() y que no expone 
        # este atributo support_masking, es un error.
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

**Attention**

Procedemos a añadir el enmascaramiento a la mitad superior de la matriz de atención para evitar que el modelo prese atención a la información del futuro, ya que solo debe usar la información del token 0 hasta el N para generar el token N+1.\
Para hacer esto, agregaremos un método **get_causal_attention_mask(self, inputs)** a nuestro TransformerDecoder para recuperar una máscara de atención que podemos pasar a nuestras capas MultiHeadAttention.

In [ ]:
def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        # Genere una matriz de forma (sequence_length, sequence_length) 
        # con 1 en una mitad y 0 en la otra
        mask = tf.cast(i >= j, dtype="int32")
        # Lo replicamos a lo largo del eje del lote para obtener una matriz 
        # de forma  (batch_size, sequence_length, sequence_length)
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

In [ ]:
def call(self, inputs, encoder_outputs, mask=None):
        # Recupera la máscara causal
        causal_mask = self.get_causal_attention_mask(inputs)
        # Prepara la máscara de entrada (que describe las 
        # ubicaciones de relleno en la secuencia de destino)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            # Fusiona las dos máscaras juntas
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            # Pasamos la máscara causal a la primera capa de atención, 
            # que realiza la self-attention sobre la secuencia de destino.
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            # Pasamos la máscara combinada a la segunda 
            # capa de atención, que relaciona la secuencia 
            # de origen con la secuencia de destino
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

# **Transformer Extremo-a-Extremo**

Para el objetivo que tenemos, decidimos utilizar un Transformer ya que es lo más adecuado en el aprendizaje de secuencia-a-secuencia.\
La atención neuronal que usan estos modelos, les permite procesar con éxito secuencias más largas y complejas que las que se manejan en los RNN.

Nosotros, cuando traducimos texto no leemos el texto palabra por palabra, manteniendo su significado y generando la traducción palabra a palabra en español.\
Lo que hacemos es alternar entre la oración original y la traducción simultaneamente, prestando atención al contexto y a las diferentes palabras.\
Eso es exactamente lo que podemos lograr con atención neuronal y Transformers.

Ya tenemos todo listo para montar el Transformer y entrenarlo

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
dense_dim = 128
num_heads = 2

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
# Codificamos la oración fuente
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
# Codificamos la oración objetivo y la combinamos con la oración fuente codificada
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
# Predecimos una palabra para cada posición de salida
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
33/33 [==============================] - 61s 2s/step - loss: 5.9637 - accuracy: 0.1963 - val_loss: 5.8579 - val_accuracy: 0.2155
Epoch 2/30
33/33 [==============================] - 43s 1s/step - loss: 5.5316 - accuracy: 0.2338 - val_loss: 5.7010 - val_accuracy: 0.2270
Epoch 3/30
33/33 [==============================] - 41s 1s/step - loss: 5.1791 - accuracy: 0.2642 - val_loss: 5.6063 - val_accuracy: 0.2462
Epoch 4/30
33/33 [==============================] - 42s 1s/step - loss: 4.8593 - accuracy: 0.2931 - val_loss: 5.5489 - val_accuracy: 0.2366
Epoch 5/30
33/33 [==============================] - 48s 1s/step - loss: 4.5745 - accuracy: 0.3190 - val_loss: 5.5006 - val_accuracy: 0.2452
Epoch 6/30
33/33 [==============================] - 43s 1s/step - loss: 4.3090 - accuracy: 0.3453 - val_loss: 5.4364 - val_accuracy: 0.2532
Epoch 7/30
33/33 [==============================] - 43s 1s/step - loss: 4.0631 - accuracy: 0.3695 - val_loss: 5.4389 - val_accuracy: 0.2507
Epoch 8/30
33/33 [==

Observamos que conseguimos, para los datos de validación, unos valores de precisión bastante bajos para nuestro modelo.\
Esto se puede deber al hecho de contar con un dataset pequeño y en el que cada oración trata un tema diferente

Traducimos nuevas oraciones en inglés nunca antes vistas con nuestro modelo

In [ ]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 30

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        # Muestra el siguiente token
        sampled_token_index = np.argmax(predictions[0, i, :])
        # Convertimos la siguiente predicción del token en una 
        # cadena y la agregamos a la oración generada
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        # Condición de salida
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = test_pairs['sourceText']
for _ in range(20):
    input_sentence = test_eng_texts.sample(n=1)

    #imprimir fila seleccionada
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
1125    British troops occupied the city on 6 April.
Name: sourceText, dtype: object
[start] este se dedicó a la ciudad natal en el condado de abril de harvard [end]
-
1110    In 1971, Douglas became a foundation lecturer ...
Name: sourceText, dtype: object
[start] en 1971 llegó a ser profesora de bibliotecología de estudios de bibliotecología de bibliotecología de la universidad de la universidad de las indias occidentales donde se fijó en la universidad
-
579    She has also been accused of apostasy which is...
Name: sourceText, dtype: object
[start] tiene más conocido como el caso de papua new guinea lo que pidieran [end]
-
1228    Rachel Mary Parsons was born in 1885, to Sir C...
Name: sourceText, dtype: object
[start] rachel mary parsons 1885–1956 ingeniera en el bank of the town donde abordó las políticas principales cirujanas de la hija mayor de la hija de sangre en nepal y seguridad
-
548    Wael Younis (Arabic: وائل يونس‎, Hebrew: ואאל ...
Name: sourceText, dtype: object
[st

In [ ]:
df1['sourceText'][152]

'She also choreographed The Channel O Music Video Awards.'

Tras ver los valores obtenidos al entrenar el transformer, no nos sorprende el resultado al utilizar este para traducir nuevas oraciones, ya que, como se puede apreciar, las traducciones son bastante incompletas.

# Apilamos Encoder y Decoder

A veces es útil apilar varias capas una tras otra para aumentar el poder de representación de una red. En tal configuración, debemos obtener todas las capas
intermedias para devolver una secuencia completa de salidas.\
Lo hacemos indicando **return_sequences=True**

Debido a los resultados anteriores, decidimos crear un nuevo modelo en el que vamos a apilar varias capas tanto del codificador como del decodificador, para buscar mejores resultados

In [ ]:
inputs = keras.Input(shape=(sequence_length,), dtype="int64")
x = layers.Embedding(input_dim=vocab_size, output_dim=64)(inputs)
x = layers.LSTM(32, return_sequences=True)(x)
outputs = layers.Dense(vocab_size, activation="softmax")(x)
model = keras.Model(inputs, outputs)

In [ ]:
embed_dim = 256
dense_dim = 128
num_heads = 2


encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
# Codificamos la oración fuente
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(encoder_outputs)



decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
# Deodificamos la oración objetivo y la combinamos con la oración fuente codificada
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)

x = layers.Dropout(0.5)(x)
# Predecimos una palabra para cada posición de salida
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
33/33 [==============================] - 64s 2s/step - loss: 7.4274 - accuracy: 0.0967 - val_loss: 6.6250 - val_accuracy: 0.1275
Epoch 2/30
33/33 [==============================] - 54s 2s/step - loss: 6.4015 - accuracy: 0.1593 - val_loss: 6.2005 - val_accuracy: 0.1830
Epoch 3/30
33/33 [==============================] - 58s 2s/step - loss: 5.9495 - accuracy: 0.1968 - val_loss: 5.8958 - val_accuracy: 0.2075
Epoch 4/30
33/33 [==============================] - 62s 2s/step - loss: 5.5778 - accuracy: 0.2263 - val_loss: 5.7841 - val_accuracy: 0.2292
Epoch 5/30
33/33 [==============================] - 52s 2s/step - loss: 5.2307 - accuracy: 0.2611 - val_loss: 5.6341 - val_accuracy: 0.2370
Epoch 6/30
33/33 [==============================] - 54s 2s/step - loss: 4.9295 - accuracy: 0.2851 - val_loss: 5.5801 - val_accuracy: 0.2436
Epoch 7/30
33/33 [==============================] - 58s 2s/step - loss: 4.6259 - accuracy: 0.3165 - val_loss: 5.5370 - val_accuracy: 0.2354
Epoch 8/30
33/33 [==

Podemos comprobar que con este modelo no conseguimos mejorar los valores de pérdida y precisión, obtenemos unos valores similares

In [ ]:
import numpy as np

Traducimos nuevas oraciones con el modelo ya entrenado

In [ ]:
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 30

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])
        # Muestra el siguiente token
        sampled_token_index = np.argmax(predictions[0, i, :])
        # Convertimos la siguiente predicción del token en una 
        # cadena y la agregamos a la oración generada
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        # Condición de salida
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = test_pairs['sourceText']

input_sec = []
decode = []
for _ in range(20):
    input_sentence = test_eng_texts.sample(n=1)
    input_sec.append(input_sentence)
    decode.append(decode_sequence(input_sentence))
    #imprimir fila seleccionada
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
1218    As of 2016, she is listed as the 38th most pow...
Name: sourceText, dtype: object
[start] como de 2016 figuró en la como auxiliar en las mujeres más poderosas del mundo según forbes [end]
-
897    In 1975 he was the first Papua New Guinean to ...
Name: sourceText, dtype: object
[start] en 1975 obtuvo la licenciatura en papúa nueva guinea donde se convirtió en 1926 del programa de una carta abierta solicitando un nuevo sistema de papúa nueva york [end]
-
1399    In its beginning, the band combined reggae and...
Name: sourceText, dtype: object
[start] en sus padres se ha ganado una familia de las áreas de rock japonesa de adultos servicios multiculturales y la ciudad [end]
-
814    He also undertook research work on the anticip...
Name: sourceText, dtype: object
[start] se llegó a investigación de investigación se incorporó al convento st hilda y un grupo emprendió en la universidad de adelaida donde se graduó de África occidental en la universidad
-
47    Improving in her new 

In [ ]:
df1

,sourceText,translatedText
802,She has been a banking chief executive in her ...,[start] Se ha desempeñado como directora ejecu...
594,Her Habilitation in philosophy was completed a...,[start] Obtuvo su habilitación en filosofía en...
151,"As a professional dancer and choreographer, sh...",[start] Como bailarina y coreógrafa profesiona...
614,She has won multiple awards for her work on br...,[start] Recibió varios reconocimientos por su ...
226,He created over 40 works for his new company a...,[start] Ha creado más de 40 obras para su nuev...
...,...,...
245,His great-grandfather was a poet and a busines...,[start] Su bisabuelo fue poeta y hombre de neg...
966,Ruth-Rolland was the president of the Central ...,[start] Fue presidenta de la Cruz Roja Centroa...
1279,"O'Kane returned to Australia, where she worked...","[start] En 1982, completó su doctorado. Regres..."
839,"Later in 1997, Ármannsson was appointed the CE...","[start] Luego, en 1997, Ármannsson fue designa..."


In [ ]:
def contar_palabras(fila):
    return len(str(fila).split())

media_palabras = df1['sourceText'].apply(contar_palabras).mean()
print(media_palabras)
media_palabras2 = df1['translatedText'].apply(contar_palabras).mean()
print(media_palabras2)

20.70904146838885
24.936777702243372


## **Conclusiones del Transformer**

La máxima precisión que logramos es un 27%, a priori diríamos que no es una modelo últil. La realidad es que el conjunto de datos contaba con tan solo 1471 registros, y el modelo fue entrenado con el 70%, es decir, hemos entrenado este modelo para traducir textos con solo 1029 pares de textos en inglés y español.\
En nuestra opinión, consideramos que consigue un muy buen resultado para los pocos datos de los que disponía.

Además, hay que tener en cuenta que los datos no eran de un tema concreto, esto afecta a la hora del vocabulario. Al tratar de temas diferentes, las palabras y el contexto de estas no se repite tanto, por lo que suma dificultad.
Por otro lado, como hemos podido ver, los textos originales son bastante largos, cosa que debemos tener en cuenta para evaluar el modelo.

Ahora, nos interesa probar si con un modelo de redes neuronales recurrentes bidireccional conseguimos unos mejores resultados para los datos que tenemos.

# RNN Bidireccional

RNNs dominaron el aprendizaje de secuencia-a-secuencia antes de ser superadas por Transformer.\

Los modelos estándar de secuencia-a-secuencia funcionan "leyendo" la oración complear antes de traducirla. Esto es importante sobretodo si la tarea trata sobre idiomas, ya que en cada idioma el orden de las palabras es diferente.

Una RNN de secuencia-a-secuencia usa un codificador RNN para producir un vector que codifica la secuencia origen completa, que se usa como estado inicial para un decodificador RNN, que observaría los elementos 0...N en la secuencia objetivo e intentaría predecir el paso N+1 en la secuencia objetivo.


In [ ]:
sequence_length

30

In [ ]:
inputs = keras.Input(shape=(sequence_length,), dtype="int64")
x = layers.Embedding(input_dim=vocab_size, output_dim=128)(inputs)
x = layers.LSTM(32, return_sequences=True)(x)
outputs = layers.Dense(vocab_size, activation="softmax")(x)
model = keras.Model(inputs, outputs)

**Encoder basado en GRU**

GRU solo tiene un vector de estado único, mientras que LSTM tiene varios.

In [ ]:
embed_dim = 256
latent_dim = 128

# La oración fuente en inglés va aquí. Especificar el nombre de la 
# entrada nos permite fit() (ajustar) el modelo con un dict de entradas
source = keras.Input(shape=(None,), dtype="int64", name="english")
# No olvidemos el enmascaramiento: es fundamental en esta configuración.
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
    # Nuestra oración fuente codificada es la 
    # última salida de un GRU bidireccional.
    layers.GRU(latent_dim), merge_mode="sum")(x)

Ahora, agreguamos el decodificador, este se forma con  una capa GRU que toma como estado inicial la oración original codificada, es decir, la salida del codificador.\
Además, agregamos una capa Dense que produce para cada paso de salida una distribución de probabilidad sobre el vocabulario en español.

In [ ]:
# La oración objetivo en español va aquí
past_target = keras.Input(shape=(None,), dtype="int64", name="spanish")
# No olvidar enmascarar
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
# La oración fuente codificada sirve como el estado 
# inicial del decodificador GRU.
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
# Predice el siguiente token
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
# Modelo de extremo a extremo: mapea la oración de origen y la oración 
# de destino a la oración de destino un paso en el futuro
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

Durante el entrenamiento, el decodificador toma como entrada la secuencia entera, gracias a la naturaleza de los RNN solo tiene en cuenta los tokens de 0 a N para predecir el token N+1 en la salida, esto quiere decir que solo usa la información del pasado para predecir el futuro.


**Entrenamiento**

In [ ]:
seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
seq2seq_rnn.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
33/33 [==============================] - 49s 1s/step - loss: 8.9562 - accuracy: 0.0851 - val_loss: 7.1751 - val_accuracy: 0.0933
Epoch 2/30
33/33 [==============================] - 30s 905ms/step - loss: 6.9127 - accuracy: 0.0894 - val_loss: 6.7278 - val_accuracy: 0.0933
Epoch 3/30
33/33 [==============================] - 32s 980ms/step - loss: 6.6902 - accuracy: 0.0895 - val_loss: 6.6684 - val_accuracy: 0.0933
Epoch 4/30
33/33 [==============================] - 32s 978ms/step - loss: 6.6199 - accuracy: 0.0901 - val_loss: 6.6493 - val_accuracy: 0.0933
Epoch 5/30
33/33 [==============================] - 32s 978ms/step - loss: 6.5808 - accuracy: 0.0914 - val_loss: 6.6362 - val_accuracy: 0.0997
Epoch 6/30
33/33 [==============================] - 30s 906ms/step - loss: 6.5481 - accuracy: 0.0931 - val_loss: 6.6241 - val_accuracy: 0.0993
Epoch 7/30
33/33 [==============================] - 32s 979ms/step - loss: 6.5273 - accuracy: 0.0940 - val_loss: 6.6102 - val_accuracy: 0.0976
Ep

Se obtienen aún peores resultados, apenas un 14% de precisión, suponemos que al tratar temas diferentes el contexto de una misma palabra en diferentes oraciones cambia, así como las palabras al rededor suya.\
Por lo tanto, al buscar patrones de orden puede suponer una dificultad que haya tan pocas oraciones y además con temas muy diferentes.

Nos parece interesante comentar que este modelo no es capaz de aprender de los datos de entrenamiento ni siquiera, los anteriores conseguían una puntuación de precisión relativamente alta para los datos de entrenamiento (entre 0.7 y 0.9), es decir, se ajustaban a estos y aprendían. En cambio este modelo no está sacando características ni patrones de los datos, ya que solo consigue un 10% de precisión durante el entrenamiento.

# **Posible solución**

Una posible solución para mejorar el rendimiento tanto del Transformer con del RNN Bidireccional sería ampliando el conjunto de datos original.\
Esto podríamos hacerlo aplicando técnicas de Deep Learning Generativo vistas en el curso, es decir, creando un **generador de texto** que reciba como entradas nuestro dataset y genere nuevos textos a partir de este.\
El problema que hemos encontrado al intentar esta propuesta es que al tener tan pocas oraciones de diferentes temas no podemos inicial la generación del texto con un "promt" determinado ya que nuestro dataset no trata de un tema en concreto. Por tanto, no tiene sentido indicar un inicio para todos los textos si son de temas diferentes.\
En el caso que vimos de las reseñas de películas es lógico y coherente que todos los textos generados empezaran con "this movie", en nuestro caso no es posible.

Si tuviesemos muchas más oraciones, aunque fuesen de temas diferentes tendría más sentido aplicar este algoritmo, ya que podríamos obtener resultados interesantes, pero para ello se necesita un dataset mucho más rico.




# **Conclusiones**

Como hemos podido observar, el Transformer nos ofrece mejores resultados que una RNN bidirecional, esto se debe a que el segundo tiene una capacidad limitada a la hora de capturar patrones de dependencia a largo plazo.\
En una RNN bidireccional la información transcurre en ambos sentidos simultaneamente, lo que permite capturar cierta información del contexto de las palabras cercanas a la palabra actual. En cambio, la dependencia a largo plazo es más dificil de capturar debido al **desvanecimiento del gradiente**, esto se produce cuando se propagan gradientes a través de una gran cantidad de pasos de tiempo, lo que hace que la información sea cada vez más dificil de recuperar.

Sin embargo, el Transformer se basa en la multihead attention para capturar estos patrones de dependencia a largo plazo. Esto le permite capturar mejor la relación entre palabras distantes, y consecuentemente, ofrece un mejor rendimiento en este tipo de tareas.

Además, el Transformer puede procesar todos los elementos de una secuencia simultaneamente, mientras que en un RNN, la información debe procesarse en orden secuencial, lo que lo hace menos eficiente.

